In [ ]:
# Courtesy of https://randomrealizations.com/posts/shap-from-scratch/

import numpy as np 
from typing import Any, Callable, Iterable
from math import factorial
from itertools import chain, combinations

class ShapFromScratchExplainer():
    def __init__(self,
                 model: Callable[[np.ndarray], float], 
                 background_dataset: np.ndarray,
                 max_samples: int = None):
        self.model = model # Set model
        if max_samples: # If max samples, randomly sample a subset of the background dataset
            max_samples = min(max_samples, background_dataset.shape[0]) 
            rng = np.random.default_rng()
            self.background_dataset = rng.choice(background_dataset, 
                                                 size=max_samples, 
                                                 replace=False, axis=0)
        else: # Use the full background dataset
            self.background_dataset = background_dataset

    def shap_values(self, X: np.ndarray) -> np.ndarray:
        """Compute SHAP values for instances in DataFrame or 2D array"""
        shap_values = np.empty(X.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                shap_values[i, j] = self._compute_single_shap_value(j, X[i, :])
        return shap_values
       
    def _compute_single_shap_value(self, 
                                   feature: int,
                                   instance: np.array) -> float:
        """
        Compute a single SHAP value given feature of interest and instance
        (equation 4 in SHAP paper)
        """
        n_features = len(instance)
        shap_value = 0
        for subset in self._get_all_other_feature_subsets(n_features, feature):
            n_subset = len(subset)
            prediction_without_feature = self._subset_model_approximation(
                subset, 
                instance
            )
            prediction_with_feature = self._subset_model_approximation(
                subset + (feature,), 
                instance
            )
            factor = self._permutation_factor(n_features, n_subset)
            shap_value += factor * (prediction_with_feature - prediction_without_feature)
        return shap_value
    
    def _get_all_subsets(self, items: list) -> Iterable:
        """Generate all possible subsets"""
        return chain.from_iterable(combinations(items, r) for r in range(len(items)+1))
    
    def _get_all_other_feature_subsets(self, n_features, feature_of_interest):
        """Generate all subsets of features excluding the feature of interest"""
        all_other_features = [j for j in range(n_features) if j != feature_of_interest]
        return self._get_all_subsets(all_other_features)

    def _permutation_factor(self, n_features, n_subset):
        return (
            factorial(n_subset) 
            * factorial(n_features - n_subset - 1) 
            / factorial(n_features) 
        )
    
    def _subset_model_approximation(self, 
                                    feature_subset: tuple[int, ...], 
                                    instance: np.array) -> float:
        """ 
        Approximate subset model prediction  (Equation 11 in SHAP paper)
        \hat{f}_S(x) = E_{x_{\hat{S}}}[f_S(x)]
        for feature subset S on single instance x
        """
        masked_background_dataset = self.background_dataset.copy()
        for j in range(masked_background_dataset.shape[1]):
            if j in feature_subset:
                masked_background_dataset[:, j] = instance[j]
        conditional_expectation_of_model = np.mean(
            self.model(masked_background_dataset)
        )
        return conditional_expectation_of_model

In [8]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

X, y = load_diabetes(as_frame=False, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=42)

lin_model = LinearRegression().fit(X_train, y_train)
rfr_model = RandomForestRegressor().fit(X_train, y_train)
gbt_model = GradientBoostingRegressor().fit(X_train, y_train)

In [12]:
import shap

def compare_methods(model, X_background, X_instances):
        
    library_explainer = shap.KernelExplainer(model.predict, X_background)
    library_shap_values = library_explainer.shap_values(X_instances)

    from_scratch_explainer = ShapFromScratchExplainer(model.predict, X_background)
    from_scratch_shap_values = from_scratch_explainer.shap_values(X_instances)
    
    print(library_shap_values)
    print()
    print(from_scratch_shap_values)

    return np.allclose(library_shap_values, from_scratch_shap_values)

In [13]:
compare_methods(lin_model, X_background=X_train[:100, :], X_instances=X_test[:5, :])

100%|██████████| 5/5 [00:00<00:00, 21.42it/s]


[[ 1.07328990e+00  1.19742336e+01 -6.78307320e+00 -8.87444563e+00
  -1.05142093e+02  5.90556891e+01  3.22045528e+00  6.97888597e+00
   1.51290727e+01 -6.34130862e-01]
 [ 2.45475216e+00  1.19742336e+01  1.67692643e+01  5.83472803e+00
   3.01327907e+01 -1.28318456e+01  9.90223092e-01 -1.42787624e+01
  -2.10969996e+01 -1.31967774e+00]
 [ 1.60462154e+00 -1.29720864e+01 -5.60545632e+00 -7.53724802e+00
  -8.52852297e+01  2.03348095e+01  7.68091966e+00 -3.64993823e+00
   4.94500912e+01 -1.14829102e+00]
 [ 2.56101848e+00  1.19742336e+01  2.50125824e+01  2.81258121e+01
  -4.18483403e+01  1.41458164e+01 -8.37675211e+00  3.78024761e+01
   5.87830524e+01  2.10805665e+00]
 [ 1.16892960e-01 -1.29720864e+01 -1.44375829e+01 -3.52565521e+00
  -2.69556925e+01  2.25564993e+01  9.81302164e-02  6.97888597e+00
  -9.64263767e+00 -8.05517582e-01]]

[[ 1.07328990e+00  1.19742336e+01 -6.78307320e+00 -8.87444563e+00
  -1.05142093e+02  5.90556891e+01  3.22045528e+00  6.97888597e+00
   1.51290727e+01 -6.34130862e-

True